In [ ]:
# 필요한 라이브러리 import
from datetime import datetime
from wcwidth import wcswidth
import time

# 메뉴 및 데이터 초기화
from data import menu_list, size_price, size_list, cart_list, membership_dict, cart

# 유틸리티 함수
# yes or no
def get_yes_or_no_input(msg):
    while True:
        user_input = input('>>>' + msg + " (yes/no): ").strip().lower()
        if user_input in ['yes', 'no']:
            return user_input
        else:
            print("⚠️'yes' 또는 'no'만 입력할 수 있어요.")

#숫자검증
from utils import get_valid_number_input

# 시작 화면
from main import start

# 메뉴 선택
from menu import menu_choice

# 수량 선택
from select_quantity import size_count

# 장바구니에 추가 및 장바구니 기능 선택
from cart import cart_function

# 장바구니 상품 삭제
from cart import del_cart

# 장바구니 보기 및 결제
def show_cart():
    if not cart_list:
        print("\n장바구니가 비어 있습니다.")
        print('-' * 50)
        start()
        return
    print_cart()
    print('-' * 50)

    while True:
        ask_mul = '1. 결제하기  2. 계속 쇼핑하기  0. 종료하기 '
        choice = get_valid_number_input(ask_mul, 0, 2) ## 예외처리 (0~3)
        if choice == 1:
            total = sum(item['total_price'] for item in cart_list)
            print(f"\n총 결제 금액: {total:,}원")

            answer = get_yes_or_no_input("결제하시겠습니까? (yes/no): ") #예외처리 (yes or no)
            if answer not in ['yes', 'no']:
                print("yes 또는 no만 입력해주세요.")
                continue
            elif answer == 'no':
                print("주문이 취소되었습니다.\n")
                return start()

            # 멤버십 적립 처리
            handle_membership()
            print("결제가 완료되었습니다. 감사합니다! 🍦\n")
            print("ඞඞඞඞඞඞඞ 로딩중 ඞඞඞඞඞඞඞ")
            print("""⠀
            로딩중~~⠀⠀⠀
            。　 ඞ 。　 . •⠀⠀
    ⠀⠀⠀⠀⠀⠀⠀⠀⠀ ⠀⣀⣀⣀⣀⠀⠀⠀⠀⠀⠀
    ⠀🍨⠀⠀⠀⠀⢀⣶⣿⣿⣿⣿⣿⣿⣶⡄⠀⠀🍦
    ⠀⠀⠀⠀⠤⢀⠀⢸⠛⠋⠘⠛⠃⠙⢻⣿⣿⠀⠀⠀
    ⠀⠀⡌⠀⠀⠀⢣⢸⠀⠘⠀⠀⠘⠀⠀⢁⡹⠀⠀⠀     🍧
    ⠀⠀⠲⡖⠒⠒⡷⠀⡱⠤⣀⣁⣀⠤⠤⡈⠀⠀⠀⠀
    ⠀⠀⠀⠹⡄⡼⡀⣻⡜⠀⠀⠀⢀⠴⠦⢌⡆⠀⠀⠀🍫
    ⠀⠀⠀⠀⠙⠁⠈⠉⣷⣶⣶⣶⣾⣦⣤⠎⠀⠀⠀⠀
    ⠀⠀⠀⠀⠀⠀⠀⠀⠘⠒⠒⠈⠓⠒⠊⠀⠀⠀⠀⠀
  결제 중입니다 😍
    """)

            time.sleep(3)
            print('=' * 50)


            print("결제가 완료되었습니다. 감사합니다! 🍦\n")
            print_receipt("I Miss You Cram!🍦", cart_list)
            cart_list.clear()
            start()
            return

        elif choice == 2:
            return menu_choice()
        elif choice == 0:
            print("\n이용해주셔서 감사합니다! 좋은 하루 되세요! 👋")
            start()

# 장바구니 출력
from cart import print_cart

# 멤버십 적립 함수
def handle_membership():
    global last_membership
    while True:
            member = input("멤버십을 적립하시겠습니까? (yes/no): ").strip().lower()
            if member not in ['yes', 'no']:
                print("yes 또는 no만 입력해주세요.")
                continue

            if member == 'no':
                print("멤버십 적립을 건너뜁니다.\n")
                last_membership = False
                break

            print("멤버십 적립을 선택하셨습니다.")

            while True:
                member_ship = input("전화 번호를 11자리를 입력하세요: ").strip()
                last_membership = member_ship

                # 유효성 검사(11자리 여부, 숫자 여부)
                if len(member_ship) != 11 or not member_ship.isdigit():
                    print("전화번호가 올바르지 않습니다. 11자리 숫자만 입력해주세요.")
                    continue

                # 포인트 적립
                if member_ship in membership_dict:
                    # 기존 회원: 1포인트 추가
                    membership_dict[member_ship] += 1
                    print(f"기존 ({member_ship}) 번호로 포인트를 적립했습니다. 적립 포인트: {membership_dict[member_ship]} Point\n")
                else:
                    # 신규 회원: 포인트 1점으로 초기화
                    membership_dict[member_ship] = 1
                    print(f"새로운 ({member_ship}) 번호로 멤버십이 생성되었습니다. 적립 포인트: {membership_dict[member_ship]} Point\n")

                return

# 영수증 정렬 함수
def align_text(text, width, align='left'):
    text_width = wcswidth(text)
    space = width - text_width
    if space <= 0:
        return text
    if align == 'left':
        return text + ' ' * space
    elif align == 'right':
        return ' ' * space + text
    elif align == 'center':
        left = space // 2
        right = space - left
        return ' ' * left + text + ' ' * right

# 영수증 프린트 함수
def print_receipt(store_name, cart_list):
    print("=" * 42)
    print(align_text(store_name, 42, 'center'))
    print("-" * 42)
    now = datetime.now()
    print(f"주문일시 : {now.strftime('%Y-%m-%d %H:%M:%S')}")
    print("-" * 42)
    print(f"{align_text('메뉴', 18)}{align_text('수량', 6, 'right')}"
          f"{align_text('단가', 9, 'right')}{align_text('금액', 9, 'right')}")
    print("-" * 42)

    total = 0
    for item in cart_list:
        name = f"{item['user_menu']} ({item['user_size']})"
        qty = item['qty']
        price = item['price_per_item']
        amount = item['total_price']
        total += amount

        print(f"{align_text(name, 18)}"
              f"{align_text(str(qty), 6, 'right')}"
              f"{align_text(f'{price:,}', 9, 'right')}"
              f"{align_text(f'{amount:,}', 9, 'right')}")

    vat = int(total * 0.1)

    print("-" * 42)
    print(f"{align_text('합계', 33)}{align_text(f'{total-vat:,}', 9, 'right')}")
    print(f"{align_text('부가세(10%)', 33)}{align_text(f'{vat:,}', 9, 'right')}")
    print(f"{align_text('총 결제금액', 33)}{align_text(f'{total:,}', 9, 'right')}")
    print("-" * 42)
    if last_membership :
      print(f'적립번호 뒷자리:{last_membership[-4:]}  적립포인트:{membership_dict[last_membership]} Point')
    else:
      print('    membership 번호가 없어서 아ship😭')

    print("=" * 42)
    print("""⠀⠀⠀⠀
            。　 ඞ 。　 . •⠀⠀
    ⠀⠀⠀⠀⠀⠀⠀⠀⠀ ⠀⣀⣀⣀⣀⠀⠀⠀⠀⠀⠀
    ⠀🍨⠀⠀⠀⠀⢀⣶⣿⣿⣿⣿⣿⣿⣶⡄⠀⠀🍦
    ⠀⠀⠀⠀⠤⢀⠀⢸⠛⠋⠘⠛⠃⠙⢻⣿⣿⠀⠀⠀
    ⠀⠀⡌⠀⠀⠀⢣⢸⠀⠘⠀⠀⠘⠀⠀⢁⡹⠀⠀⠀     🍧
    ⠀⠀⠲⡖⠒⠒⡷⠀⡱⠤⣀⣁⣀⠤⠤⡈⠀⠀⠀⠀
    ⠀⠀⠀⠹⡄⡼⡀⣻⡜⠀⠀⠀⢀⠴⠦⢌⡆⠀⠀⠀🍫
    ⠀⠀⠀⠀⠙⠁⠈⠉⣷⣶⣶⣶⣾⣦⣤⠎⠀⠀⠀⠀
    ⠀⠀⠀⠀⠀⠀⠀⠀⠘⠒⠒⠈⠓⠒⠊⠀⠀⠀⠀⠀
  결제 감사합니다 😍 보고싶으니까 또와유~
    """)
    print("=" * 42)

# main()
start()

보고싶었어요! I Miss You Cram!🍦

📋 메뉴판

1. 초코
2. 바닐라
3. 딸기
4. 우유
5. 말차
6. 민트초코



원하시는 메뉴를 선택해주세요. (1~6, 0:처음으로):  1


--------------------------------------------------

📏 사이즈 선택
1. S - 2000원
2. M - 3000원
3. L - 4000원


사이즈를 선택해주세요 (1~3):  1
수량을 입력해주세요 (1~10):  1



🍦 장바구니에 추가되었습니다! 🍦

🛒 현재 장바구니:
1. 초코 (S) x1 = 2000원
--------------------------------------------------


...


🛒 현재 장바구니:
1. 초코 (S) x1 = 2000원
--------------------------------------------------


1. 결제하기  2. 계속 쇼핑하기  0. 종료하기  1



총 결제 금액: 2,000원


결제하시겠습니까? (yes/no):  yes
멤버십 번호(전화번호 11자리):  01056557968


01056557968님 적립 완료! 총 1 포인트

            I Miss You Cram!🍦            
------------------------------------------
주문일시 : 2025-06-11 09:59:24
------------------------------------------
메뉴                수량     단가     금액
------------------------------------------
⠀⠀⠀⠀
            。　 ඞ 。　 . •⠀⠀
    ⠀⠀⠀⠀⠀⠀⠀⠀⠀ ⠀⣀⣀⣀⣀⠀⠀⠀⠀⠀⠀
    ⠀🍨⠀⠀⠀⠀⢀⣶⣿⣿⣿⣿⣿⣿⣶⡄⠀⠀🍦
    ⠀⠀⠀⠀⠤⢀⠀⢸⠛⠋⠘⠛⠃⠙⢻⣿⣿⠀⠀⠀
    ⠀⠀⡌⠀⠀⠀⢣⢸⠀⠘⠀⠀⠘⠀⠀⢁⡹⠀⠀⠀     🍧
    ⠀⠀⠲⡖⠒⠒⡷⠀⡱⠤⣀⣁⣀⠤⠤⡈⠀⠀⠀⠀
    ⠀⠀⠀⠹⡄⡼⡀⣻⡜⠀⠀⠀⢀⠴⠦⢌⡆⠀⠀⠀🍫
    ⠀⠀⠀⠀⠙⠁⠈⠉⣷⣶⣶⣶⣾⣦⣤⠎⠀⠀⠀⠀
    ⠀⠀⠀⠀⠀⠀⠀⠀⠘⠒⠒⠈⠓⠒⠊⠀⠀⠀⠀⠀
  결제 감사합니다 😍 보고싶으니까 또와유~
            
------------------------------------------
초코 (S)               1    2,000    2,000
------------------------------------------
합계                                 1,800
부가세(10%)                            200
총 결제금액                          2,000
------------------------------------------
적립번호 뒷자리:7968  적립포인트:1 Point
보고싶었어요! I Miss You Cram!🍦

📋 메뉴판

1. 초코
2. 바닐라
3. 딸기
4. 우유
5. 말차
6. 민트초코
